In [1]:
import sys
import xarray as xr
import numpy as np
import pandas as pd
%matplotlib inline
# import wrf


# import personal modules
# Path to modules
sys.path.append('../modules')
# Import my modules

In [2]:
path_to_data = '/expanse/nfs/cw3e/cwp140/'
path_to_out  = '../out/'       # output files (numerical results, intermediate datafiles) -- read & write
path_to_figs = '../figs/'      # figures

In [3]:
## load ar dates within baja region
fname = '../out/bbox_dates_baja.csv'
df = pd.read_csv(fname)
df = df.set_index(pd.to_datetime(df['datetime']))
ar_dates = df['datetime'].values
## create year month columns in dataframe 
df['yearmonth'] = df.index.strftime("%Y%m")
new = df.drop_duplicates('yearmonth')
new

,Unnamed: 0,datetime,yearmonth
datetime,,,
2003-02-25 09:00:00,0,2003-02-25 09:00:00,200302
2003-03-16 05:00:00,21,2003-03-16 05:00:00,200303
2003-09-09 09:00:00,65,2003-09-09 09:00:00,200309
2004-09-03 22:00:00,92,2004-09-03 22:00:00,200409
2004-12-29 14:00:00,120,2004-12-29 14:00:00,200412
...,...,...,...
2013-07-26 06:00:00,364,2013-07-26 06:00:00,201307
2000-02-24 09:00:00,0,2000-02-24 09:00:00,200002
2010-03-07 08:00:00,449,2010-03-07 08:00:00,201003


In [ ]:
%%time
## create a smaller list of files to open
ds_lst = []
for index, row in new.iterrows():
    print(row['yearmonth'])
    fname = path_to_data + 'preprocessed/ERA5/ivt/{0}_IVT.nc'.format(row['yearmonth'])
    # fname_lst.append(fname)
    idx = (df['yearmonth'] == row['yearmonth'])
    ar_dates = df.loc[idx]['datetime'].values
    ds = xr.open_dataset(fname)
    ds = ds.sel(time=ar_dates)
    ds = ds.mean(dim='time')
    ds_lst.append(ds)

200302
200303
200309
200409
200412
200501
200502
200504
200509
200709
200711
200712
200801
200810
200812
201001
201009
201110
201309
201404
201409
201506
201511
201611
201612
201702
201903
201905
202003
202102
202112
202301
202212
202303
200210
200401
200312
200402
200410
200603
200609
200912
201002
201012
201112
201210
201301
201310
201701
201709
201810
202103
202108
202211
200510
200608
200610
200701
200802
200902
201102
201408
201911
202011
202310
200010
200310
201006
201411
201609
202006
200102
200104
200209
200311
200602
200911
201403
201502
201503
201505
200904
200910
201004
200906
200304
200408
200809
200909
201211
201808
201902
200811
200004
200006
200901
200211
201209
201402
200305
201311
201412
201507
200009
201904
200805
201601
200008
200710
201509
202105
202206
201111
201212
202209
201011
201703
202308
202307
200511
201602
201803
201806
201305
200703
201401
200108
200803
201510
202012
200201
201201
201410
201711
202110
200101
201608


In [ ]:
test = xr.concat(ds_lst, pd.Index(new['yearmonth'].values, name="yearmonth"))
test = test.mean(dim='yearmonth')

out_fname = '../out/baja_IVT_composite.nc'
test.to_netcdf(path=out_fname, mode = 'w', format='NETCDF4')

In [ ]:
# %%time

# ## load ERA5 IVT
# def preprocess(ds):
#     return ds.sel(lat=slice(50., 15.), lon=slice(-180., -80.))
    
# ds = xr.open_mfdataset(fname_lst, engine='netcdf4', combine='by_coords', preprocess=preprocess, parallel=True)
# ds